# 2CS SIL2-SIQ2 Lab04. Naïve Bayes

<p style='text-align: right;font-style: italic;'>Designed by: Dr. Abdelkrime Aries</p>

In this lab, we will learn about two generative models:
- Naïve Bayes

**Team:**
- **Member 01**: Mecherak Thanina
- **Member 02**: Rezazi Mohammed Abdessamed
- **Group**: SIL2

In [2]:
import sys, timeit
from typing          import Tuple, List, Type
from collections.abc import Callable

sys.version

'3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]'

In [3]:
import numpy             as np
import pandas            as pd 
import matplotlib.pyplot as plt 
import matplotlib
%matplotlib inline

np.__version__, pd.__version__, matplotlib.__version__

('1.24.3', '2.0.3', '3.7.2')

In [4]:
import sklearn

from sklearn.naive_bayes   import CategoricalNB
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics       import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection         import train_test_split
from sklearn.naive_bayes             import MultinomialNB, GaussianNB
from sklearn.linear_model            import LogisticRegression
from sklearn.tree                    import DecisionTreeClassifier
from sklearn.metrics                 import precision_score, recall_score
import timeit


sklearn.__version__

'1.3.0'

## I. Algorithms implementation

In this section, we will try to implement multinomial Naive Bayes.


**>> Try to use "numpy" which will save a lot of time and effort**

In [5]:
# Dataset play 

# outlook & temperature & humidity & windy
Xplay = np.array([
    ['sunny'   , 'hot' , 'high'  , 'no'],
    ['sunny'   , 'hot' , 'high'  , 'yes'],
    ['overcast', 'hot' , 'high'  , 'no'],
    ['rainy'   , 'mild', 'high'  , 'no'],
    ['rainy'   , 'cool', 'normal', 'no'],
    ['rainy'   , 'cool', 'normal', 'yes'],
    ['overcast', 'cool', 'normal', 'yes'],
    ['sunny'   , 'mild', 'high'  , 'no'],
    ['sunny'   , 'cool', 'normal', 'no'],
    ['rainy'   , 'mild', 'normal', 'no'],
    ['sunny'   , 'mild', 'normal', 'yes'],
    ['overcast', 'mild', 'high'  , 'yes'],
    ['overcast', 'hot' , 'normal', 'no'],
    ['rainy'   , 'mild', 'high'  , 'yes']
])

Yplay = np.array([
    'no', 
    'no', 
    'yes', 
    'yes', 
    'yes', 
    'no', 
    'yes', 
    'no', 
    'yes', 
    'yes', 
    'yes', 
    'yes', 
    'yes', 
    'no'
])

len(Xplay), len(Yplay)

(14, 14)

### I.1. Prior probability

Given an output list $Y$, the probability of each class $c_k$ is estimated as:
$$p(c_k) = \frac{|\{y / y \in Y \text{ et } y = c_k\}|}{|Y|}$$

Our function must return two lists:
- One containing the names of unique classes.
- Another containing probabilities of unique classes of the first list.

In [7]:
def fit_prior(Y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]: 
    cls  = np.unique(Y) # vocabulary
    prior = []
    for c in cls:
        prior.append(np.log(np.count_nonzero(Y==c)/ len(Y)))
    return cls, prior

#=====================================================================
# UNIT TEST
#=====================================================================
# Result: 
# (array(['no', 'yes'], dtype='<U3'), array([-1.02961942, -0.44183275]))
#---------------------------------------------------------------------

fit_prior(Yplay)

(array(['no', 'yes'], dtype='<U3'), [-1.0296194171811581, -0.4418327522790392])

### I.2. Multinomial Likelihood probability

Given:
- $A$: a categorical feature
- $V$: unique values of this feature (feature's categories)
- $Y$: the ouput
- $C$: the classes
- $\alpha$: smoothing factor

calculate the log likelihood:
$$ \log p(A=v|Y=c_k) = \log(|\{ y \in Y / y = c_k \text{ and } A = v\}| + \alpha) - \log(|\{y = c_k\}| + \alpha * |V|)$$

The function must:
- add a token "\<UNK\>" to $V$
- return feature's categories (vocabulary) $V$
- return a matrix where rows are $V$ and colums are $C$ containing the log probability $p(V|C)$
- when alpha=0 and v=UNK, we will have an error, in this case we will consider log probability as $-\infty$
- when the probability is 0, we will consider log probability as $-\infty$

In [11]:
# TODO: Multinomial Likelihood probability
def fit_likelihood(A: np.ndarray, Y: np.ndarray, C: np.ndarray, alpha: float = 0.) -> Tuple[np.ndarray, np.ndarray]: 
    V = np.append(np.unique(A), '<UNK>')  
    likelihood = np.zeros((len(V), len(C))) 

    for i, v in enumerate(V):  
        for j, c in enumerate(C):  
            count = np.sum((A == v) & (Y == c))  
            total = np.sum(Y == c)  
            if count == 0 and alpha == 0:
                likelihood[i, j] = -np.inf
            else:
                likelihood[i, j] = np.log((count + alpha) / (total + alpha * len(V)))

    return V, likelihood







#=====================================================================
# UNIT TEST
#=====================================================================
# Result: 
#  ((array(['overcast', 'rainy', 'sunny', '<UNK>'], dtype='<U8'), 
#  array(     [[ -inf, -0.81093022],
#             [-0.91629073, -1.09861229], 
#             [-0.51082562, -1.5040774 ], 
#             [ -inf, -inf]])), 
# (array(['overcast', 'rainy', 'sunny', '<UNK>'], dtype='<U8'), 
#  
   #array([[-2.19722458, -0.95551145], 
#              [-1.09861229, -1.178655 ], 
#              [-0.81093022, -1.46633707], 
#              [-2.19722458, -2.56494936]])))
#---------------------------------------------------------------------
C_t = np.array(['no', 'yes'])
fit_likelihood(Xplay[:, 0], Yplay, C_t), fit_likelihood(Xplay[:, 0], Yplay, C_t, alpha=1)

((array(['overcast', 'rainy', 'sunny', '<UNK>'], dtype='<U8'),
  array([[       -inf, -0.81093022],
         [-0.91629073, -1.09861229],
         [-0.51082562, -1.5040774 ],
         [       -inf,        -inf]])),
 (array(['overcast', 'rainy', 'sunny', '<UNK>'], dtype='<U8'),
  array([[-2.19722458, -0.95551145],
         [-1.09861229, -1.178655  ],
         [-0.81093022, -1.46633707],
         [-2.19722458, -2.56494936]])))

### I.3. Model training

**Nothing to code here, although you have to know how it functions for next use**

Given a feature $X_j$, a value $v \in X_j$ and a class $c_k$, the likeelihood is calculated:
$$ P(X_j=v|y=c_k) = \frac{|\{ y \in Y / y = c_k \text{ and } X_j = v\}|}{|\{y = c_k\}|}$$

This function aims to generate parameters $\theta$. 
In our case, paramters are diffrent from those of *logistic regrssion*.
They are a dictionary (map) with two entries:
- "prior" key having a dictionary as value, having "v" a list of values and "p" a list of their respective probabilities.
- "likelihood" a list of dictinaries reprsnting statistics of each feature (the same order of $X$ features)

In [12]:
def fit_NB(X: np.ndarray, 
        Y: np.ndarray,
        alpha=0.
        ) -> object: 
    
    Theta   = {'prior': {}, 'likelihood': []}

    Theta['prior']['v'], Theta['prior']['p'] = fit_prior(Y)

    for j in range(X.shape[1]): 
        likelihood = {}
        likelihood['v'], likelihood['p'] = fit_likelihood(X[:, j], Y, Theta['prior']['v'], alpha=alpha)
        Theta['likelihood'].append(likelihood)
    
    return Theta


#=====================================================================
# UNIT TEST
#=====================================================================
# Result: 
# {'prior': {'v': array(['no', 'yes'], dtype='<U3'), 
#    'p': array([-1.02961942, -0.44183275])}, 
#    'likelihood': [{'v': array(['overcast', 'rainy', 'sunny', '<UNK>'], dtype='<U8'), 
#    'p': array([[-2.19722458, -0.95551145], 
#                     [-1.09861229, -1.178655 ], 
#                     [-0.81093022, -1.46633707], 
#                     [-2.19722458, -2.56494936]])}, 
#     {'v': array(['cool', 'hot', 'mild', '<UNK>'], dtype='<U8'), 
#       'p': array([[-1.5040774 , -1.178655 ], 
#                        [-1.09861229, -1.46633707], 
#                        [-1.09861229, -0.95551145], 
#                        [-2.19722458, -2.56494936]])}, 
#     {'v': array(['high', 'normal', '<UNK>'], dtype='<U8'), 
#       'p': array([[-0.47000363, -1.09861229], 
#                        [-1.38629436, -0.5389965 ], 
#                        [-2.07944154, -2.48490665]])}, 
#     {'v': array(['no', 'yes', '<UNK>'], dtype='<U8'), 
#       'p': array([[-0.98082925, -0.5389965 ], 
#                        [-0.69314718, -1.09861229], 
#                        [-2.07944154, -2.48490665]])}]}
#---------------------------------------------------------------------
Theta_play = fit_NB(Xplay, Yplay, alpha=1)

Theta_play

{'prior': {'v': array(['no', 'yes'], dtype='<U3'),
  'p': [-1.0296194171811581, -0.4418327522790392]},
 'likelihood': [{'v': array(['overcast', 'rainy', 'sunny', '<UNK>'], dtype='<U8'),
   'p': array([[-2.19722458, -0.95551145],
          [-1.09861229, -1.178655  ],
          [-0.81093022, -1.46633707],
          [-2.19722458, -2.56494936]])},
  {'v': array(['cool', 'hot', 'mild', '<UNK>'], dtype='<U8'),
   'p': array([[-1.5040774 , -1.178655  ],
          [-1.09861229, -1.46633707],
          [-1.09861229, -0.95551145],
          [-2.19722458, -2.56494936]])},
  {'v': array(['high', 'normal', '<UNK>'], dtype='<U8'),
   'p': array([[-0.47000363, -1.09861229],
          [-1.38629436, -0.5389965 ],
          [-2.07944154, -2.48490665]])},
  {'v': array(['no', 'yes', '<UNK>'], dtype='<U8'),
   'p': array([[-0.98082925, -0.5389965 ],
          [-0.69314718, -1.09861229],
          [-2.07944154, -2.48490665]])}]}

### I.4. Multinomial prediction
Let's examine prediction equation: 
$$\hat{c} = \arg\max\limits_{c_k} \log P(y=c_k) + \sum\limits_{f_j \in \overrightarrow{f}} \log P(f_j|y=c_k)$$

Our goal is to calculate approximate probabilities of all classes given a sample like indicated in the next equation:
$$P(y=c_k) + \sum\limits_{f_j \in \overrightarrow{f}} \log P(f_j|y=c_k)$$

- Given one sample $x$, we have to return a list of probabilities. 
- If prior=false, we must not consider prior probabilities.

In [17]:
# You can use this function in the next implimentation
# It takes a list of unique values V and a given value v
# It returns the position of v in V
# If v does not exist in V, it rturns -1
def find_idx(V: np.ndarray, v: str) -> int:
    k = np.argwhere(V == v).flatten()
    if len(k):
        return k[0]
    return -1

# TODO: Prediction
def predict_NB_1(Xi    : np.ndarray, 
            Theta: object,  
            add_prior: bool  = True
           ) -> List[float]: 

    return None

#=====================================================================
# UNIT TEST
#=====================================================================
# Result: 
# (array([-6.81036293, -5.8230459 ]), array([-4.68213123, -3.99491878]))

#---------------------------------------------------------------------

X_t = np.array([
    ['rainy', 'cool', 'normal', 'yes'],
    ['snowy', 'cool', 'normal', 'yes'],
    ['sunny', 'hot' , 'normal', 'no']
])

predict_NB_1(X_t[1, :], Theta_play), predict_NB_1(X_t[0, :], Theta_play, add_prior=False)

(None, None)

### I.5. Final product

**>> Nothing to code here**


In [18]:
class OurCategoricalNB(object): 
        
    def fit(self, X, Y, alpha=0.):
        self.Theta = fit_NB(X, Y, alpha=alpha)
    
    def predict(self, X, add_prior=True, prob=False): 
        Y_pred = []
        for i in range(len(X)): 
            Y_pred.append(predict_NB_1(X[i,:], self.Theta, add_prior=add_prior))
        
        Y_pred = np.array(Y_pred)

        if prob:
            return Y_pred

        return np.choose(np.argmax(Y_pred, axis=1), self.Theta['prior']['v'])

#=====================================================================
# UNIT TEST
#=====================================================================
# Result: 
# (array(['yes', 'yes', 'yes'], dtype='<U3'),
#  array([[-5.9348942 , -3.11351531],
#         [-6.22257627, -3.68887945],
#         [-5.73009978, -3.32993436]]))
#---------------------------------------------------------------------

cnb = OurCategoricalNB()
cnb.fit(Xplay, Yplay, alpha=1)
X_t = np.array([
    ['rainy', 'cool', 'normal', 'yes'],
    ['snowy', 'cool', 'normal', 'yes'],
    ['sunny', 'hot' , 'normal', 'no']
])

cnb.predict(X_t), cnb.predict(X_t, prob=True)

AxisError: axis 1 is out of bounds for array of dimension 1

## II. Application and Analysis

In this section, we will test different concepts by running an experiment, formulating a hypothesis and trying to justify it.

### II.1. Prior probability 

We want to test the effect of prior probability.
To do this, we trained two models:
1. With prior probability
1. Without prior probability (It considers a uniform distribution of classes)

To test whether the models have adapted well to the training dataset, we will test them on the same dataset and calculate the classification ratio.


In [19]:
nb_withPrior     = CategoricalNB(alpha=1.0, fit_prior=True )
nb_noPrior       = CategoricalNB(alpha=1.0, fit_prior=False)

enc         = OrdinalEncoder()
Xplay_tf    = enc.fit_transform(Xplay)
nb_withPrior.fit(Xplay_tf, Yplay)
nb_noPrior.fit(Xplay_tf, Yplay)

Ypred_withPrior = nb_withPrior.predict(Xplay_tf)
Ypred_noPrior = nb_noPrior.predict(Xplay_tf)


print( 'Considring prior probability'  )
print(classification_report(Yplay, Ypred_withPrior))

print( 'No prior probability'  )
print(classification_report(Yplay, Ypred_noPrior))

Considring prior probability
              precision    recall  f1-score   support

          no       1.00      0.80      0.89         5
         yes       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93      0.93        14

No prior probability
              precision    recall  f1-score   support

          no       0.67      0.80      0.73         5
         yes       0.88      0.78      0.82         9

    accuracy                           0.79        14
   macro avg       0.77      0.79      0.78        14
weighted avg       0.80      0.79      0.79        14



**TODO: Analyze the results**

1. What do you notice, indicating if prior probability is useful in this case?
1. How does this probability affect the outcome?
1. When are we sure that using this probability is unnecessary? 

**Answer**

1. From the results, it’s clear that the model considering prior probability has a higher overall accuracy (93%) compared to the model that doesn’t (79%). This indicates that in this case, prior probability is useful for improving the model’s performance


1. Prior probability provides an indication of the overall likelihood of a class in the dataset. When considered, it can help guide the model’s predictions, especially in cases where the data is imbalanced. In this case, it appears to have helped the model generalize better, as evidenced by the higher accuracy.

1. The use of prior probability might be less useful when the classes are perfectly balanced, i.e., there is an equal number of samples for each class. In this case, the prior probability for each class would be the same and wouldn’t provide additional information to the model. Also, if the features allow for a clear separation of classes, prior probability might have less impact on the model’s performance. However, it’s generally recommended to include it, as it can help improve the model’s robustness.

### II.2. Smoothing

We want to test the Lidstone smoothing's effect.
To do this, we trained three models:
1. alpha = 1 (Laplace smoothing)
1. alpha = 0.5
1. alpha = 0 (without smoothing)

In [11]:
NBC_10 = CategoricalNB(alpha = 1.0 )
NBC_05 = CategoricalNB(alpha = 0.5 )
NBC_00 = CategoricalNB(alpha = 0.0 )

NBC_10.fit( Xplay_tf,   Yplay )
NBC_05.fit( Xplay_tf,   Yplay )
NBC_00.fit( Xplay_tf,   Yplay )

Y_10   = NBC_10.predict(Xplay_tf)
Y_05   = NBC_05.predict(Xplay_tf)
Y_00   = NBC_00.predict(Xplay_tf)


print(          'Alpha = 1.0'             )
print(classification_report(Yplay, Y_10))

print(          'Alpha = 0.5'             )
print(classification_report(Yplay, Y_05))

print(          'Alpha = 0.0'             )
print(classification_report(Yplay, Y_00))


Alpha = 1.0
              precision    recall  f1-score   support

          no       1.00      0.80      0.89         5
         yes       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93      0.93        14

Alpha = 0.5
              precision    recall  f1-score   support

          no       1.00      0.80      0.89         5
         yes       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93      0.93        14

Alpha = 0.0
              precision    recall  f1-score   support

          no       1.00      0.80      0.89         5
         yes       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93     

/opt/env/ml/lib/python3.10/site-packages/sklearn/naive_bayes.py:1504: RuntimeWarning: divide by zero encountered in log
  np.log(smoothed_cat_count) - np.log(smoothed_class_count.reshape(-1, 1))


**TODO: Analyze the results**

1. What do you notice, indicating if smoothing affects performance in this case?
1. Based on the past answeer, Why? 
1. Why do we get a "RuntimeWarning: divide by zero" error? 
1. What is the benefit of smoothing (generally; not just for this case)?

**Answer**

1. it appears that the smoothing parameter alpha does not significantly affect the performance of the model in this specific case. The accuracy, precision, recall, and f1-score remain the same for all three models (with alpha values of 1.0, 0.5, and 0.0). .


1.  in our case , all categories in the test set (which is the same as our training set) are already represented in the training set. Therefore, there are no instances of zero-frequency problem. This is why changing the alpha value does not affect the performance of your model.


1.  this warning can occur when the model encounters a category in the test data that was not present in the training data, and no smoothing has been applied. When the model tries to calculate the probability of this category, it involves a division by its count in the training data. Since the category was not in the training data, its count is zero, leading to a division by zero


1. The general benefit of smoothing is to prevent zero probabilities in probabilistic models like Naive Bayes. By ensuring that no category has a zero probability, smoothing allows the model to handle unseen data more effectively. This can lead to more robust and accurate predictions when the model is applied to new, unseen data. It’s particularly useful in domains like natural language processing, where the vocabulary can be vast and the training data might not cover all possible words

### II.3. Naive Bayes performance

Naive Bayes is known to generate powerful models when it comes to classifying textual documents.
We want to test this proposition using spam detection over [SMS Spam Collection Dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset) dataset.

Each message is represented using term frequency (TF), where a word is considered as a feature.
In this case, a message is represented by a vector of frequencies (how many times each word appeared in the message).
We want to compare these models:
1. Multinomial Naive Bayes (MNB)
1. Gaussian Naive Bayes (GNB)
1. Logistic Regression (LR) 
1. Decision Tree (DT)

In [12]:
# reading the dataset
messages = pd.read_csv('data/spam.csv', encoding='latin-1')
# renaming features: text and class
messages = messages.rename(columns={'v1': 'class', 'v2': 'text'})
# keeping only these two features
messages = messages.filter(['text', 'class'])

messages.head()

,text,class
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [13]:
models = [
    MultinomialNB(),
    GaussianNB(),
    LogisticRegression(solver='lbfgs'),
    #solver=sag is slower; so I chose the fastest
    DecisionTreeClassifier()
]

algos = [
    'Multinomial Naive Bayes (MNB)', 
    'Gaussian Naive Bayes  (GNB)', 
    'Logistic Regression (LR)', 
    'Decision Tree (DT)'
]

perf = {
    'train_time': [],
    'test_time' : [],
    'recall'    : [],
    'precision' : []
}


msg_train, msg_test, Y_train, Y_test = train_test_split(messages['text'] ,
                                                        messages['class'],
                                                        test_size    = 0.2, 
                                                        random_state = 0  )

count_vectorizer = CountVectorizer()
X_train          = count_vectorizer.fit_transform(msg_train).toarray()
X_test           = count_vectorizer.transform    (msg_test ).toarray()


for model in models:
    # ==================================
    # TRAIN 
    # ==================================
    start_time = timeit.default_timer()
    model.fit(X_train, Y_train)
    perf['train_time'].append(timeit.default_timer() - start_time)
    
    # ==================================
    # TEST 
    # ==================================
    start_time = timeit.default_timer()
    Y_pred     = model.predict(X_test)
    perf['test_time'].append(timeit.default_timer() - start_time)
    
    # ==================================
    # PERFORMANCE 
    # ==================================
    # In here, we are interrested in "spam" class which is our positive class
    perf['precision'].append(precision_score(Y_test, Y_pred, pos_label='spam'))
    perf['recall'   ].append(recall_score   (Y_test, Y_pred, pos_label='spam'))

    
pd.DataFrame({
    'Algorithm' : algos,
    'Train time': perf['train_time'],
    'Test time' : perf['test_time'],
    'Precision' : perf['precision'],
    'Recall'    : perf['recall']
})

,Algorithm,Train time,Test time,Precision,Recall
0,Multinomial Naive Bayes (MNB),0.559776,0.029081,0.987179,0.927711
1,Gaussian Naive Bayes (GNB),0.551694,0.127377,0.616667,0.891566
2,Logistic Regression (LR),0.555998,0.028857,0.986111,0.855422
3,Decision Tree (DT),13.080956,0.016129,0.909677,0.849398


**TODO: Analyze the results**

1. What do you notice about training time? (order the algorithms)
1. Why did we get these results based on the algorithms? (discuss each algorithm with respect to training time)
1. What do you notice about the testing time? (order the algorithms)
1. Why did we get these results based on the algorithms? (discuss each algorithm with respect to testing time)
1. Why is the Gaussian model less efficient than the multinomial based on the nature of the two algorithms?
1. Why is the Gaussian model less efficient than the multinomial based on the nature of the problem/data?

**Answer**

1. based on the results : here’s how the algorithms rank from fastest to slowest:

Logistic Regression (LR): With a training time of approximately 0.556 seconds, this is the fastest algorithm among the ones you tested.
Multinomial Naive Bayes (MNB): This algorithm took approximately 0.560 seconds to train, making it the second fastest.
Gaussian Naive Bayes (GNB): With a training time of approximately 0.552 seconds, this is the third fastest algorithm.
Decision Tree (DT): This algorithm took significantly longer to train than the others, with a training time of approximately 13.081 seconds. It is the slowest among the tested algorithms



1.Multinomial Naive Bayes (MNB): This algorithm is fast because it treats features independently, which simplifies the computation. It’s particularly efficient with discrete data, like word counts for text classification.
Gaussian Naive Bayes (GNB): Similar to MNB, GNB assumes feature independence, making it computationally efficient. However, it’s designed for continuous data and may require additional time to estimate the mean and variance of each class.
Logistic Regression (LR): LR uses a numerical optimization technique to find the best parameters, which can be computationally intensive, especially for high-dimensional data. However, it’s still relatively fast compared to more complex models.
Decision Tree (DT): DTs can be slower to train because they require more complex computations to determine the best splits. The training time can increase significantly with the number of features and depth of the tree.


1. algorithms rank from fastest to slowest:

Decision Tree (DT): With a testing time of approximately 0.016 seconds, this is the fastest algorithm among the ones you tested.
Logistic Regression (LR): This algorithm took approximately 0.029 seconds to test, making it the second fastest.
Multinomial Naive Bayes (MNB): This algorithm took approximately 0.029 seconds to test, making it the third fastest.
Gaussian Naive Bayes (GNB): With a testing time of approximately 0.127 seconds, this is the slowest among the tested algorithms.


1. Decision Tree (DT): Decision Trees are fast at making predictions once they are trained. The testing phase involves traversing the tree from root to leaf, which is computationally efficient.
Logistic Regression (LR) and Multinomial Naive Bayes (MNB): Both these algorithms are relatively fast at making predictions. They involve simple mathematical operations on the feature values and model parameters.
Gaussian Naive Bayes (GNB): This algorithm might take longer in the testing phase compared to others because it needs to calculate the probability of a feature value given a class using the Gaussian distribution, which can be more computationally intensive


1. The Multinomial Naive Bayes model is designed for discrete data. It works well with data that can be expressed as counts, such as the frequency of words in text. This makes it a good choice for text classification tasks, and it can be more efficient when dealing with this type of data.

On the other hand, the Gaussian Naive Bayes model is designed for continuous data. It assumes that the data for each label is drawn from a Gaussian distribution. While this is a good assumption for many types of continuous data, it may not be as efficient for discrete data like word counts. The Gaussian model needs to calculate the mean and standard deviation for each class, which can be computationally intensive.

So, while both models are useful, the Multinomial Naive Bayes model can be more efficient for tasks like text classification. The Gaussian Naive Bayes model might be a better choice for tasks involving continuous data


1. The Multinomial Naive Bayes model is designed for discrete count data, like the frequency of words in a text. It’s particularly efficient when dealing with this type of data because it simply counts the occurrences of features and calculates probabilities.

On the other hand, the Gaussian Naive Bayes model is designed for continuous data and assumes that the data for each class is drawn from a Gaussian distribution. This involves calculating the mean and standard deviation for each feature, which can be computationally intensive. Therefore, when applied to discrete count data (like word counts), the Gaussian model can be less efficient than the Multinomial model.

So, the nature of your data and the problem you’re trying to solve can determine which model is more efficient. For text classification problems with count data, the Multinomial Naive Bayes model is often a better choice. For problems involving continuous data, the Gaussian Naive Bayes model might be more suitable.

In [14]:
print("  _____    __                                              _               ")
print(" |_   _|  / _|                                            | |              ")
print("   | |   | |_     _   _    ___    _   _      __ _    ___  | |_             ")
print("   | |   |  _|   | | | |  / _ \  | | | |    / _` |  / _ \ | __|            ")
print("  _| |_  | |     | |_| | | (_) | | |_| |   | (_| | |  __/ | |_             ")
print(" |_____| |_|      \__, |  \___/   \__,_|    \__, |  \___|  \__|            ")
print("                   __/ |                     __/ |                         ")
print("                  |___/                     |___/                          ")
print("  _     _       _            __                                            ")
print(" | |   | |     (_)          / _|                 _                         ")
print(" | |_  | |__    _   ___    | |_    __ _   _ __  (_)                        ")
print(" | __| | '_ \  | | / __|   |  _|  / _` | | '__|                            ")
print(" | |_  | | | | | | \__ \   | |   | (_| | | |     _                         ")
print("  \__| |_| |_| |_| |___/   |_|    \__,_| |_|    ( )                        ")
print("                                                |/                         ")
print("                                                                           ")
print("                                                                           ")
print("                                                                           ")
print("  _   _    ___    _   _      __ _   _ __    ___                            ")
print(" | | | |  / _ \  | | | |    / _` | | '__|  / _ \                           ")
print(" | |_| | | (_) | | |_| |   | (_| | | |    |  __/                           ")
print("  \__, |  \___/   \__,_|    \__,_| |_|     \___|                           ")
print("   __/ |                                                                   ")
print("  |___/                                                                    ")
print("                    _                                                __    ")
print("                   | |                                            _  \ \   ")
print("  _ __     ___     | |__    _   _   _ __ ___     __ _   _ __     (_)  | |  ")
print(" | '_ \   / _ \    | '_ \  | | | | | '_ ` _ \   / _` | | '_ \         | |  ")
print(" | | | | | (_) |   | | | | | |_| | | | | | | | | (_| | | | | |    _   | |  ")
print(" |_| |_|  \___/    |_| |_|  \__,_| |_| |_| |_|  \__,_| |_| |_|   (_)  | |  ")
print("                                                                     /_/   ")
print("                                                                           ")

  _____    __                                              _               
 |_   _|  / _|                                            | |              
   | |   | |_     _   _    ___    _   _      __ _    ___  | |_             
   | |   |  _|   | | | |  / _ \  | | | |    / _` |  / _ \ | __|            
  _| |_  | |     | |_| | | (_) | | |_| |   | (_| | |  __/ | |_             
 |_____| |_|      \__, |  \___/   \__,_|    \__, |  \___|  \__|            
                   __/ |                     __/ |                         
                  |___/                     |___/                          
  _     _       _            __                                            
 | |   | |     (_)          / _|                 _                         
 | |_  | |__    _   ___    | |_    __ _   _ __  (_)                        
 | __| | '_ \  | | / __|   |  _|  / _` | | '__|                            
 | |_  | | | | | | \__ \   | |   | (_| | | |     _                         
  \__| |_| |